In [19]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [24]:
pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
import shap

In [29]:
from sklearn.inspection import permutation_importance
from sklearn.utils.validation import check_is_fitted

In [31]:
df = pd.read_csv("C:\\Users\\jithi\\Desktop\\lung_cancer_data.csv")

In [32]:
df.head()

,Patient_ID,Age,Gender,Smoking_History,Years_Smoked,Pack_Years,Family_History_Cancer,Occupation,Exposure_to_Toxins,Residential_Area,...,Previous_Cancer_Diagnosis,Tumor_Size_cm,Metastasis_Status,Stage_of_Cancer,Treatment_Type,Survival_Years,Follow_Up_Visits,Medication_Response,Symptom_Progression,Year_of_Diagnosis
0,1,69,Male,Never,30,3,False,Farmer,False,Urban,...,True,11.02,True,III,Surgery,12,24,Good,Stable,2007
1,2,32,Female,Former,6,61,False,Office Worker,False,Urban,...,False,14.29,True,II,Chemotherapy,6,12,Poor,Stable,2009
2,3,89,Male,Never,2,9,True,Office Worker,True,Rural,...,False,9.47,False,III,Chemotherapy,6,15,Good,Worsening,2015
3,4,78,Female,Never,11,69,False,Factory Worker,True,Urban,...,False,2.22,False,IV,Chemotherapy,13,25,Moderate,Improving,2012
4,5,38,Male,Former,11,57,False,Farmer,False,Rural,...,False,8.26,False,III,Palliative,3,4,Good,Stable,2014


In [35]:
df.columns

Index(['Patient_ID', 'Age', 'Gender', 'Smoking_History', 'Years_Smoked',
       'Pack_Years', 'Family_History_Cancer', 'Occupation',
       'Exposure_to_Toxins', 'Residential_Area', 'BMI',
       'Lung_Function_Test_Result', 'Chest_Pain_Symptoms',
       'Shortness_of_Breath', 'Chronic_Cough', 'Weight_Loss',
       'Physical_Activity_Level', 'Dietary_Habits', 'Air_Quality_Index',
       'Comorbidities', 'Previous_Cancer_Diagnosis', 'Tumor_Size_cm',
       'Metastasis_Status', 'Stage_of_Cancer', 'Treatment_Type',
       'Survival_Years', 'Follow_Up_Visits', 'Medication_Response',
       'Symptom_Progression', 'Year_of_Diagnosis'],
      dtype='object')

In [37]:
df.shape

(300000, 30)

In [39]:
unwanted_columns = ['Patient_ID', 'Occupation', 'Residential_Area']

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 30 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Patient_ID                 300000 non-null  int64  
 1   Age                        300000 non-null  int64  
 2   Gender                     300000 non-null  object 
 3   Smoking_History            300000 non-null  object 
 4   Years_Smoked               300000 non-null  int64  
 5   Pack_Years                 300000 non-null  int64  
 6   Family_History_Cancer      300000 non-null  bool   
 7   Occupation                 300000 non-null  object 
 8   Exposure_to_Toxins         300000 non-null  bool   
 9   Residential_Area           300000 non-null  object 
 10  BMI                        300000 non-null  float64
 11  Lung_Function_Test_Result  300000 non-null  float64
 12  Chest_Pain_Symptoms        300000 non-null  bool   
 13  Shortness_of_Breath        30

In [43]:
df.isnull().sum()

Patient_ID                       0
Age                              0
Gender                           0
Smoking_History                  0
Years_Smoked                     0
Pack_Years                       0
Family_History_Cancer            0
Occupation                       0
Exposure_to_Toxins               0
Residential_Area                 0
BMI                              0
Lung_Function_Test_Result        0
Chest_Pain_Symptoms              0
Shortness_of_Breath              0
Chronic_Cough                    0
Weight_Loss                      0
Physical_Activity_Level          0
Dietary_Habits                   0
Air_Quality_Index                0
Comorbidities                90376
Previous_Cancer_Diagnosis        0
Tumor_Size_cm                    0
Metastasis_Status                0
Stage_of_Cancer                  0
Treatment_Type                   0
Survival_Years                   0
Follow_Up_Visits                 0
Medication_Response              0
Symptom_Progression 

In [45]:
categorical_cols = ['Gender', 'Smoking_History', 'Occupation', 'Residential_Area',
                    'Physical_Activity_Level', 'Dietary_Habits', 'Comorbidities',
                    'Stage_of_Cancer', 'Treatment_Type', 'Medication_Response', 'Symptom_Progression']

In [47]:
label_encoders = {}
categorical_columns = df.select_dtypes(include=['object']).columns
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le


In [48]:
df.head(10)

,Patient_ID,Age,Gender,Smoking_History,Years_Smoked,Pack_Years,Family_History_Cancer,Occupation,Exposure_to_Toxins,Residential_Area,...,Previous_Cancer_Diagnosis,Tumor_Size_cm,Metastasis_Status,Stage_of_Cancer,Treatment_Type,Survival_Years,Follow_Up_Visits,Medication_Response,Symptom_Progression,Year_of_Diagnosis
0,1,69,1,2,30,3,False,1,False,2,...,True,11.02,True,2,3,12,24,0,1,2007
1,2,32,0,1,6,61,False,2,False,2,...,False,14.29,True,1,0,6,12,2,1,2009
2,3,89,1,2,2,9,True,2,True,0,...,False,9.47,False,2,0,6,15,0,2,2015
3,4,78,0,2,11,69,False,0,True,2,...,False,2.22,False,3,0,13,25,1,0,2012
4,5,38,1,1,11,57,False,1,False,0,...,False,8.26,False,2,1,3,4,0,1,2014
5,6,41,0,0,9,67,False,3,False,2,...,False,5.72,False,3,0,1,39,2,2,2020
6,7,20,0,0,29,9,False,2,False,0,...,False,1.05,False,1,0,18,34,0,1,2002
7,8,39,0,2,49,61,True,0,True,1,...,False,14.50,False,0,2,11,26,1,2,2015
8,9,70,0,2,2,49,False,2,True,1,...,True,5.03,True,1,2,3,22,0,2,2016
9,10,19,0,1,31,73,False,1,True,0,...,False,0.72,False,0,0,18,19,1,1,2022


In [55]:
scaler = StandardScaler()
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

In [57]:
X = df.drop(columns=[target] + drop_columns)
y = df[target]

NameError: name 'target' is not defined

In [61]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)


NameError: name 'X' is not defined

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)


NameError: name 'X_resampled' is not defined

In [79]:
# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


NameError: name 'X_train' is not defined

In [103]:
rf_model = RandomForestClassifier(class_weight='balanced', random_state=42)
rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict(X_test)
print("Random Forest Performance:\n", classification_report(y_test, rf_preds))


NameError: name 'X_train' is not defined

In [101]:
rf_preds = rf_model.predict(X_test)

NameError: name 'rf_model' is not defined

In [97]:
cm = confusion_matrix(y_test, rf_preds)

NameError: name 'confusion_matrix' is not defined

In [99]:
def confusion_matrix(y_test, y_pred, model_name="Model"):
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(cm)
    disp.plot(cmap='Blues', values_format='d')
    plt.title(f"Confusion Matrix - {model_name}")
    plt.show()

In [81]:
# Train models
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000)
}


NameError: name 'XGBClassifier' is not defined

In [83]:
# Evaluate models
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"\n{name} Model Evaluation:")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))
    cm = confusion_matrix(y_test, y_pred)
    ConfusionMatrixDisplay(cm).plot()
    plt.show()

NameError: name 'models' is not defined

In [85]:
# Explain model predictions using SHAP
explainer = shap.Explainer(models["Random Forest"], X_train)
shap_values = explainer(X_test[:100])
shap.summary_plot(shap_values, X_test[:100])


NameError: name 'shap' is not defined